In [ ]:
%matplotlib inline


# Calculating the initial piece of coastal and along-coast connectivity

There are multiple analysis steps to get to various perspectives on coastal connectivity, but for all of them, this is the first step.

## Making coastal paths

In [ ]:
# script to make paths

In [ ]:
# show some plots

## Running analysis on coastal paths

In [ ]:
# where does this analysis run and what does it look like and do

## Where this is being used